In [4]:
!pip list


Package                       Version
----------------------------- ---------------
absl-py                       2.1.0
aiobotocore                   2.7.0
aiohttp                       3.9.0
aioitertools                  0.7.1
aiosignal                     1.2.0
alabaster                     0.7.12
anaconda-anon-usage           0.4.2
anaconda-catalogs             0.2.0
anaconda-client               1.12.1
anaconda-cloud-auth           0.1.4
anaconda-navigator            2.5.2
anaconda-project              0.11.1
anyio                         3.5.0
appdirs                       1.4.4
archspec                      0.2.1
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arrow                         1.2.3
astroid                       2.14.2
astropy                       5.3.4
asttokens                     2.0.5
astunparse                    1.6.3
async-lru                     2.0.4
atomicwrites                  1.4.0
attrs                         23.1.0
Automat  

In [3]:
!pip install opencv-python matplotlib imageio gdown tensorflow


Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for opencv-python from https://files.pythonhosted.org/packages/c7/ec/9dabb6a9abfdebb3c45b0cc52dec901caafef2b2c7e7d6a839ed86d81e91/opencv_python-4.9.0.80-cp37-abi3-win_amd64.whl.metadata
  Using cached opencv_python-4.9.0.80-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Obtaining dependency information for gdown from https://files.pythonhosted.org/packages/9e/db/c5dad38733f0995dea4480de38df1b3f6222b77b0dc89aad2402d86e2ff4/gdown-5.0.1-py3-none-any.whl.metadata
  Using cached gdown-5.0.1-py3-none-any.whl.metadata (5.6 kB)
  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/93/21/9b035a4f823d6aee2917c75415be9a95861ff3d73a0a65e48edbf210cec1/tensorflow-2.15.0-cp311-cp311-win_amd64.whl.metadata
  Using cached tensorflow-2.15.0-cp311-cp311-win_amd64.whl.metadata (3.6 kB)
  Obtaining dependency information for tensorflow-intel==2.15.0 from http

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


   ----- --------------------------------- 42.7/300.9 MB 770.3 kB/s eta 0:05:36
   ----- --------------------------------- 42.7/300.9 MB 767.5 kB/s eta 0:05:37
   ----- --------------------------------- 42.7/300.9 MB 769.3 kB/s eta 0:05:36
   ----- --------------------------------- 42.8/300.9 MB 768.5 kB/s eta 0:05:36
   ----- --------------------------------- 42.8/300.9 MB 768.4 kB/s eta 0:05:36
   ----- --------------------------------- 42.8/300.9 MB 771.1 kB/s eta 0:05:35
   ----- --------------------------------- 42.9/300.9 MB 771.2 kB/s eta 0:05:35
   ----- --------------------------------- 42.9/300.9 MB 770.2 kB/s eta 0:05:36
   ----- --------------------------------- 42.9/300.9 MB 768.4 kB/s eta 0:05:36
   ----- --------------------------------- 42.9/300.9 MB 767.5 kB/s eta 0:05:37
   ----- --------------------------------- 43.0/300.9 MB 766.6 kB/s eta 0:05:37
   ----- --------------------------------- 43.0/300.9 MB 764.8 kB/s eta 0:05:38
   ----- -------------------------------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for opencv-python from https://files.pythonhosted.org/packages/c7/ec/9dabb6a9abfdebb3c45b0cc52dec901caafef2b2c7e7d6a839ed86d81e91/opencv_python-4.9.0.80-cp37-abi3-win_amd64.whl.metadata
  Using cached opencv_python-4.9.0.80-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Obtaining dependency information for gdown from https://files.pythonhosted.org/packages/9e/db/c5dad38733f0995dea4480de38df1b3f6222b77b0dc89aad2402d86e2ff4/gdown-5.0.1-py3-none-any.whl.metadata
  Using cached gdown-5.0.1-py3-none-any.whl.metadata (5.6 kB)
  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/93/21/9b035a4f823d6aee2917c75415be9a95861ff3d73a0a65e48edbf210cec1/tensorflow-2.15.0-cp311-cp311-win_amd64.whl.metadata
  Using cached tensorflow-2.15.0-cp311-cp311-win_amd64.whl.metadata (3.6 kB)
  Obtaining dependency information for tensorflow-intel==2.15.0 from http

In [8]:
!pip install --upgrade tensorflow


Defaulting to user installation because normal site-packages is not writeable


In [9]:
import os
import cv2
import tensorflow as tf
import numpy as np 
from typing import List
from matplotlib import pyplot as plt
import imageio


In [11]:
physical_devices = tf.config.list_physical_devices('GPU')
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    pass

In [12]:
tf.config.list_physical_devices('GPU')

[]

In [14]:
#Build data loading fuctions
import gdown

In [15]:
url = 'https://drive.google.com/uc?id=1YlvpDLix3S-U8fd-gqRwPcWXAXm8JwjL'
output = 'data.zip'
gdown.download(url,output, quiet=False)
gdown.extractall('data.zip')

Downloading...
From (original): https://drive.google.com/uc?id=1YlvpDLix3S-U8fd-gqRwPcWXAXm8JwjL
From (redirected): https://drive.google.com/uc?id=1YlvpDLix3S-U8fd-gqRwPcWXAXm8JwjL&confirm=t&uuid=a8935506-3d6c-4d43-813f-0d61f06ce602
To: C:\Users\baafi\Documents\Projects\Lip_ml\lip_reading_app\data.zip
100%|███████████████████████████████████████████████████████████████████████████████| 423M/423M [01:53<00:00, 3.72MB/s]


['data/',
 'data/alignments/',
 'data/alignments/s1/',
 'data/alignments/s1/bbaf2n.align',
 'data/alignments/s1/bbaf3s.align',
 'data/alignments/s1/bbaf4p.align',
 'data/alignments/s1/bbaf5a.align',
 'data/alignments/s1/bbal6n.align',
 'data/alignments/s1/bbal7s.align',
 'data/alignments/s1/bbal8p.align',
 'data/alignments/s1/bbal9a.align',
 'data/alignments/s1/bbas1s.align',
 'data/alignments/s1/bbas2p.align',
 'data/alignments/s1/bbas3a.align',
 'data/alignments/s1/bbaszn.align',
 'data/alignments/s1/bbaz4n.align',
 'data/alignments/s1/bbaz5s.align',
 'data/alignments/s1/bbaz6p.align',
 'data/alignments/s1/bbaz7a.align',
 'data/alignments/s1/bbbf6n.align',
 'data/alignments/s1/bbbf7s.align',
 'data/alignments/s1/bbbf8p.align',
 'data/alignments/s1/bbbf9a.align',
 'data/alignments/s1/bbbm1s.align',
 'data/alignments/s1/bbbm2p.align',
 'data/alignments/s1/bbbm3a.align',
 'data/alignments/s1/bbbmzn.align',
 'data/alignments/s1/bbbs4n.align',
 'data/alignments/s1/bbbs5s.align',
 'data/al

In [ ]:
def load_video(path:str) -> List[float]:
cap = cv2.VideoCapture(path)
frames = []
    